In [14]:
# Auto-reloading for functions during development
%load_ext autoreload
%autoreload 2

# Import necessary packages
import pandas as pd
import numpy as np
import os
import torch
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import pearsonr
from SALib.sample import saltelli, fast_sampler
from SALib.analyze import sobol, fast, rbd_fast
import torch.nn as nn

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:
# Load Data
X = pd.read_csv("./simulation_data/MaterialConfigurations_1024.csv")
Y_rom_files = {
    "ROM Flexion": "./simulation_data/ROM_Results_Flexion_wMoment_1024.csv",
    "ROM Extension": "./simulation_data/ROM_Results_Extension_wMoment_1024.csv",
    "ROM Lateral Bending": "./simulation_data/ROM_Results_LateralBending_wMoment_1024.csv",
    "ROM Axial Rotation": "./simulation_data/ROM_Results_AxialRotation_wMoment_1024.csv"
}
Y_idp_files = {
    "IDP Flexion": "./simulation_data/IDP_Results_Flexion_wMoment_1024.csv",
    "IDP Extension": "./simulation_data/IDP_Results_Extension_wMoment_1024.csv",
    "IDP Lateral Bending": "./simulation_data/IDP_Results_LateralBending_wMoment_1024.csv",
    "IDP Axial Rotation": "./simulation_data/IDP_Results_AxialRotation_wMoment_1024.csv"
}

# Linear model-based SA methods

In [47]:
# Helper Functions
def load_last_column(file_path):
    """Loads the last column of a CSV file as a Series."""
    data = pd.read_csv(file_path)
    return data.iloc[:, -1]

def scale_data(df):
    """Applies min-max scaling to a DataFrame."""
    return (df - df.min()) / (df.max() - df.min())

def perform_regression(X, y):
    """Performs linear regression and returns model, coefficients, intercept, and R² score."""
    model = LinearRegression()
    model.fit(X, y)
    y_pred = model.predict(X)
    r2 = r2_score(y, y_pred)
    return model, model.coef_, model.intercept_, r2

def calculate_r2_for_parameters(X, y):
    """Calculates R² for each parameter individually."""
    r2_scores = {}
    for param in X.columns:
        model = LinearRegression()
        model.fit(X[[param]], y)
        r2_scores[param] = model.score(X[[param]], y)
    return r2_scores

def calculate_pearson_correlations(X, y):
    """Calculates Pearson's correlation and p-value for each parameter."""
    correlations, p_values = {}, {}
    for param in X.columns:
        correlation, p_value = pearsonr(X[param], y)
        correlations[param] = correlation
        p_values[param] = p_value
    return correlations, p_values

def load_car_scores(filename):
    """Loads CAR scores from a CSV file."""
    return pd.read_csv(filename).iloc[:, 0].values

def save_dict_to_csv(data, filename, index_name="Parameter"):
    df = pd.DataFrame(data)
    df.index.name = index_name
    df.to_csv(filename)  

In [56]:
# Scale Data
X_scaled = scale_data(X)
Y_rom = {name: scale_data(load_last_column(path)) for name, path in Y_rom_files.items()}
Y_idp = {name: scale_data(load_last_column(path)) for name, path in Y_idp_files.items()}

# Linear Regression Analysis
results = {}
for name, y in {**Y_rom, **Y_idp}.items():
    model, coefs, intercept, r2 = perform_regression(X_scaled, y)
    r2_params = calculate_r2_for_parameters(X_scaled, y)
    correlation, _ = calculate_pearson_correlations(X_scaled, y)
    results[name] = {
        "model": model,
        "coefficients": coefs,
        "intercept": intercept,
        "r2_score": r2,
        "r2_params": r2_params,
        "pearson_correlation": correlation
    }

# Load CAR Scores for Comparison
car_score_files = {
    "ROM Flexion": "./results_data/CARScores_ROM_flexion.csv",
    "ROM Extension": "./results_data/CARScores_ROM_extension.csv",
    "ROM Lateral Bending": "./results_data/CARScores_ROM_lateral_bending.csv",
    "ROM Axial Rotation": "./results_data/CARScores_ROM_axial_rotation.csv",
    "IDP Flexion": "./results_data/CARScores_IDP_flexion.csv",
    "IDP Extension": "./results_data/CARScores_IDP_extension.csv",
    "IDP Lateral Bending": "./results_data/CARScores_IDP_lateral_bending.csv",
    "IDP Axial Rotation": "./results_data/CARScores_IDP_axial_rotation.csv"
}
car_scores = {name: load_car_scores(path) for name, path in car_score_files.items()}

# Summary Ratios
for name in results:
    r2_total = results[name]["r2_score"]
    car_scores_squared = car_scores[name]**2
    results[name]["r2_ratios"] = {param: r2 / r2_total for param, r2 in results[name]["r2_params"].items()}
    results[name]["car_ratios"] = car_scores_squared / r2_total

# Results contain all necessary information for further analysis or visualization
print("Analysis completed. Results are stored in the 'results' dictionary.")

Analysis completed. Results are stored in the 'results' dictionary.


In [59]:
# Ensure the results directory exists
output_dir = "./results_data/"
os.makedirs(output_dir, exist_ok=True)

# Extract and store CAR²-ratios, R² ratios, and Pearson's correlation coefficients
car_ratios_data = {}
r2_ratios_data = {}
pearson_correlations_data = {}

# Populate dictionaries with extracted results for each response variable
for name, result in results.items():
    car_ratios_data[name] = result["car_ratios"]
    r2_ratios_data[name] = result["r2_ratios"]
    pearson_correlations_data[name] = result["pearson_correlation"]

# Convert dictionaries to DataFrames and save as CSV files
save_dict_to_csv(car_ratios_data, os.path.join(output_dir, "CAR_squared_ratios.csv"))
save_dict_to_csv(r2_ratios_data, os.path.join(output_dir, "COD_ratios.csv"))
save_dict_to_csv(pearson_correlations_data, os.path.join(output_dir, "Pearson_correlation_coefficients.csv"))

print("Files saved successfully to './results_data/' directory.")

Files saved successfully to './results_data/' directory.


# Variance-based SA methods

In [ ]:
# Device setup for neural network operations
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters for the neural network
hparams = {
    'input_dim': 15,
    'output_dim': 1,
    'num_units': 256,
    'num_layers': 5,
    'dropout_p': 0
}

# Neural Network Model Definition
class RegressionNet(nn.Module):
    def __init__(self, hparams):
        super(RegressionNet, self).__init__()
        layers = []
        for i in range(hparams['num_layers'] - 1):
            i_dim = hparams['input_dim'] if i == 0 else hparams['num_units'] // (2 ** (i - 1))
            o_dim = hparams['num_units'] // (2 ** i)
            layers.append(nn.Linear(i_dim, o_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(p=hparams['dropout_p']))
        layers.append(nn.Linear(o_dim, hparams['output_dim']))
        self.layers = nn.ModuleList(layers)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

# Load Pretrained Neural Network Models
def load_nn_model(file_path, hparams):
    model = RegressionNet(hparams)
    model.load_state_dict(torch.load(file_path, map_location=device))
    model.eval()
    return model

# Paths to neural network models
nn_model_rom = load_nn_model('./models/nn_model_ROM.pth', hparams)
nn_model_idp = load_nn_model('./models/nn_model_IDP.pth', hparams)

Sobol analysis

In [ ]:
# Sobol Analysis Setup
problem = {
    'num_vars': X.shape[1],  # Number of input variables,
    'names': X.columns.tolist(),  # Input variable names
    'bounds': [[0, 1]] * X.shape[1]  # Normalized bounds
}

num_samples = 2**13  # To reach convergence
param_values = saltelli.sample(problem, num_samples, calc_second_order=True)

# Generate Load Cases for Sobol Analysis
def generate_load_cases(base_params):
    load_cases = {
        "flexion": np.concatenate([np.zeros((base_params.shape[0], 1)), np.ones((base_params.shape[0], 1)) * 5, base_params], axis=1),
        "extension": np.concatenate([np.ones((base_params.shape[0], 1)) * 1/3, np.ones((base_params.shape[0], 1)) * 5, base_params], axis=1),
        "lateral_bending": np.concatenate([np.ones((base_params.shape[0], 1)) * 2/3, np.ones((base_params.shape[0], 1)) * 5, base_params], axis=1),
        "axial_rotation": np.concatenate([np.ones((base_params.shape[0], 1)), np.ones((base_params.shape[0], 1)) * 5, base_params], axis=1)
    }
    return {case: torch.tensor(params, dtype=torch.float32, device=device) for case, params in load_cases.items()}

param_values_torch = generate_load_cases(param_values)

# Predictions Using Neural Network Models
def predict_with_model(model, param_values):
    predictions = {}
    with torch.no_grad():
        for case, params in param_values.items():
            predictions[case] = model(params).cpu().numpy().squeeze()
    return predictions

# Perform Sobol Analysis
def perform_sobol_analysis(problem, predictions):
    sobol_results = {}
    for case, preds in predictions.items():
        Si = sobol.analyze(problem, preds, num_resamples=1000, print_to_console=False)
        sobol_results[case] = Si
    return sobol_results

# Store Sobol Results
def sobol_results_to_csv(sobol_results, metric):
    output_dir = './results_data/'
    os.makedirs(output_dir, exist_ok=True)
    
    for case, Si in sobol_results.items():
        # Single-order and total-order indices
        df_1d = pd.DataFrame({
            'S1': Si['S1'],
            'S1_conf': Si['S1_conf'],
            'ST': Si['ST'],
            'ST_conf': Si['ST_conf'],
        }, index=problem['names'])
        
        # Second-order indices
        df_s2 = pd.DataFrame(Si['S2'], index=problem['names'], columns=problem['names'])
        df_s2_conf = pd.DataFrame(Si['S2_conf'], index=problem['names'], columns=problem['names'])
        
        # Paths for saving the results with "sobol" in the filename
        path_s1st = f"{output_dir}SobolS1ST_{metric}_{case}.csv"
        path_s2 = f"{output_dir}SobolS2_{metric}_{case}.csv"
        path_s2_conf = f"{output_dir}SobolS2conf_{metric}_{case}.csv"
        
        # Save dataframes to CSV
        df_1d.to_csv(path_s1st)
        df_s2.to_csv(path_s2)
        df_s2_conf.to_csv(path_s2_conf)

# Execute Sobol Analysis for ROM and IDP Metrics
rom_predictions = predict_with_model(nn_model_rom, param_values_torch)
idp_predictions = predict_with_model(nn_model_idp, param_values_torch)

rom_sobol_results = perform_sobol_analysis(problem, rom_predictions)
idp_sobol_results = perform_sobol_analysis(problem, idp_predictions)

# Save results to CSV files
sobol_results_to_csv(rom_sobol_results, "ROM")
sobol_results_to_csv(idp_sobol_results, "IDP")

print("Sobol analysis completed. Results saved to './results_data/'.")

eFAST analysis

In [20]:
# Define eFAST Analysis Problem Setup
problem = {
    'num_vars': X.shape[1],  # Number of input variables,
    'names': X.columns.tolist(),  # Input variable names
    'bounds': [[0, 1]] * X.shape[1]  # Normalized bounds
}

# Number of samples for eFAST
M_fast = 4
num_samples = 2**13  # Ensures sufficient convergence

# Generate eFAST Samples
param_values = fast_sampler.sample(problem, num_samples, M=M_fast, seed=4)

# Generate Load Cases for eFAST Analysis
def generate_load_cases(base_params):
    """Generates parameters for each load case."""
    load_cases = {
        "flexion": np.concatenate([np.zeros((base_params.shape[0], 1)), np.ones((base_params.shape[0], 1)) * 5, base_params], axis=1),
        "extension": np.concatenate([np.ones((base_params.shape[0], 1)) * 1/3, np.ones((base_params.shape[0], 1)) * 5, base_params], axis=1),
        "lateral_bending": np.concatenate([np.ones((base_params.shape[0], 1)) * 2/3, np.ones((base_params.shape[0], 1)) * 5, base_params], axis=1),
        "axial_rotation": np.concatenate([np.ones((base_params.shape[0], 1)), np.ones((base_params.shape[0], 1)) * 5, base_params], axis=1)
    }
    return {case: torch.tensor(params, dtype=torch.float32, device=device) for case, params in load_cases.items()}

# Device setup for neural network operations
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load cases for eFAST
param_values_torch = generate_load_cases(param_values)

# Predictions Using Neural Network Models
def predict_with_model(model, param_values):
    """Generates predictions for each load case using the neural network model."""
    predictions = {}
    with torch.no_grad():
        for case, params in param_values.items():
            predictions[case] = model(params).cpu().numpy().squeeze()
    return predictions

# Perform eFAST Analysis
def perform_fast_analysis(problem, predictions):
    """Performs eFAST analysis and stores results for each load case."""
    fast_results = {}
    for case, preds in predictions.items():
        Si = fast.analyze(problem, preds, M=M_fast, num_resamples=1000, conf_level=0.95, print_to_console=False, seed=4)
        fast_results[case] = Si
    return fast_results

# Store eFAST Results
def fast_results_to_csv(fast_results, metric):
    """Converts and saves eFAST analysis results to CSV files."""
    output_dir = './results_data/'
    os.makedirs(output_dir, exist_ok=True)
    
    for case, Si in fast_results.items():
        # First-order and total-order indices
        df_1d = pd.DataFrame({
            'S1': Si['S1'],
            'S1_conf': Si['S1_conf'],
            'ST': Si['ST'],
            'ST_conf': Si['ST_conf'],
        }, index=problem['names'])
        
        # Path for saving the results
        path_s1st = f"{output_dir}EFAST_{metric}_{case}.csv"
        
        # Save dataframe to CSV
        df_1d.to_csv(path_s1st)

# Execute eFAST Analysis for ROM and IDP Metrics
rom_predictions = predict_with_model(nn_model_rom, param_values_torch)
idp_predictions = predict_with_model(nn_model_idp, param_values_torch)

rom_fast_results = perform_fast_analysis(problem, rom_predictions)
idp_fast_results = perform_fast_analysis(problem, idp_predictions)

# Save results to CSV files
fast_results_to_csv(rom_fast_results, "ROM")
fast_results_to_csv(idp_fast_results, "IDP")

print("eFAST analysis completed. Results saved to './results_data/'.")

eFAST analysis completed. Results saved to './results_data/'.


RBD-FAST

In [62]:
# Define the RBD-FAST Analysis Problem Setup
problem = {
    'num_vars': X.shape[1],  # Number of input variables,
    'names': X.columns.tolist(),  # Input variable names
    'bounds': [[0, 1]] * X.shape[1]  # Normalized bounds
}

# Load response variables for ROM and IDP
Y_rom = {case: pd.read_csv(path).iloc[:, -1] for case, path in Y_rom_files.items()}
Y_idp = {case: pd.read_csv(path).iloc[:, -1] for case, path in Y_idp_files.items()}

# Perform RBD-FAST Analysis
def perform_rbd_fast_analysis(problem, X, y_dict):
    """Performs RBD-FAST analysis for each response variable in different load cases."""
    rbd_fast_results = {}
    for case, y in y_dict.items():
        Si = rbd_fast.analyze(problem, X=X.values, Y=y.values, print_to_console=False)
        rbd_fast_results[case] = Si
    return rbd_fast_results

# Store RBD-FAST Results
def rbd_fast_results_to_csv(rbd_fast_results, metric):
    """Saves RBD-FAST results to CSV files for each load case."""
    output_dir = './results_data/'
    os.makedirs(output_dir, exist_ok=True)
    
    for case, Si in rbd_fast_results.items():
        # Extract load case by removing the metric prefix ("ROM" or "IDP") and joining the rest
        load_case = "_".join(case.split()[1:]).lower().replace(" ", "")  # Preserve full load case
        
        # Create the filename with the desired format
        filename = f"RBDFAST_{metric}_{load_case}.csv"
        
        df_results = pd.DataFrame({
            'S1': Si['S1'],
            'S1_conf': Si['S1_conf']
        }, index=problem['names'])

        # Save results to the specified path
        df_results.to_csv(os.path.join(output_dir, filename))

# Execute RBD-FAST Analysis for ROM and IDP Metrics
rom_rbd_fast_results = perform_rbd_fast_analysis(problem, X, Y_rom)
idp_rbd_fast_results = perform_rbd_fast_analysis(problem, X, Y_idp)

# Save results to CSV files
rbd_fast_results_to_csv(rom_rbd_fast_results, "ROM")
rbd_fast_results_to_csv(idp_rbd_fast_results, "IDP")

print("RBD-FAST analysis completed. Results saved in './results_data/'.")

RBD-FAST analysis completed. Results saved in './results_data/'.


# Gradient-based IG method

In [71]:
# Define input feature names for the model
FEATURES = ['LoadCase', 'Moment', 'C10Nucleus', 'C01Nucleus', 'C10Annulus', 'K1Annulus', 'K2Annulus', 'Kappa', 
                    'K1Circ', 'K2Circ', 'K1Rad', 'K2Rad', 'FiberAngle', 'FiberAngleCirc', 'FiberAngleRad']

# Define load cases and their corresponding normalized values
load_cases = ['Flexion', 'Extension', 'Lateral_Bending', 'Axial_Rotation']
load_cases_assigned_values = [0, 2/3, 1, 1/3]

# Load normalized data for the IG method
df_trainval = pd.read_csv("./simulation_data/trainval_df_large_rom_norm.csv")
df_test = pd.read_csv("./simulation_data/test_df_large_rom_norm.csv")

def integrated_gradients(model, input_sample, baseline, steps=50):
    """Computes Integrated Gradients (IG) for a given input sample."""
    # Generate scaled inputs
    scaled_inputs = [(baseline + float(i) / steps * (input_sample - baseline)).unsqueeze(0)
                     for i in range(0, steps + 1)]

    # Compute gradients for each scaled input
    grads = []
    for scaled_input in scaled_inputs:
        scaled_input = scaled_input.clone().detach().requires_grad_(True)  
        output = model(scaled_input)
        grad = torch.autograd.grad(outputs=output, inputs=scaled_input, create_graph=True)[0]
        grads.append(grad)

    # Average the gradients
    avg_grads = torch.mean(torch.stack(grads), dim=0)

    # Integrated gradients approximation
    integrated_grads = (input_sample - baseline) * avg_grads

    return integrated_grads

# Function to calculate and save IG for a specific model and metric
def calculate_ig_for_metric(model, df_trainval, df_test, metric_name):
    """Calculates and stores Integrated Gradients (IG) sensitivity values for a specific metric (ROM/IDP)."""
    model.eval() # Set model to evaluation mode
    
    results = {}
    for load_case, value in zip(load_cases, load_cases_assigned_values):
        # Filter test samples for the current load case and moment value
        samples = df_test[np.isclose(df_test.LoadCase, value) & np.isclose(df_test.Moment, 1)]
        samples = samples.drop(['config_id', 'y_ROM', 'y_IDP'], axis=1)
        
        # Convert test samples to a tensor
        inputs = torch.tensor(samples.values, dtype=torch.float32, requires_grad=True)
        num_samples = len(inputs)
        
        # Compute the baseline as the mean of training/validation data for the current load case
        baseline = df_trainval[np.isclose(df_trainval.LoadCase, value) & np.isclose(df_trainval.Moment, 1)]
        baseline = baseline.drop(['config_id', 'y_ROM', 'y_IDP'], axis=1).mean().values
        baseline = torch.tensor(baseline, dtype=torch.float32)
        baseline[0] = value # Set baseline LoadCase value
        baseline[1] = 1 # Set baseline Moment value
        
        # Initialize tensor to store IG values for each sample
        integrated_grads = torch.zeros(num_samples, len(FEATURES))
        for i in range(num_samples):
            input_sample = inputs[i]
            ig = integrated_gradients(model, input_sample, baseline) # Compute IG
            integrated_grads[i] = ig
        
        # Compute the average absolute IG values across all test samples
        avg_integrated_sensitivity = torch.mean(integrated_grads.abs(), dim=0)[2:]  # Skip LoadCase and Moment
        normalized_sensitivity = avg_integrated_sensitivity / torch.sum(avg_integrated_sensitivity)
        
        # Store normalized sensitivities in the results dictionary
        results[load_case] = {FEATURES[i + 2]: normalized_sensitivity[i].item() for i in range(len(normalized_sensitivity))}
    
    # Save results to CSV
    output_dir = './results_data/'
    os.makedirs(output_dir, exist_ok=True)
    for load_case in results:
        pd.DataFrame.from_dict(results[load_case], orient='index', columns=[f'{metric_name} Sensitivity']).to_csv(
            os.path.join(output_dir, f'IG_{metric_name}_{load_case.lower()}.csv'))

# Calculate IG for ROM and IDP
calculate_ig_for_metric(nn_model_rom, df_trainval, df_test, "ROM")
calculate_ig_for_metric(nn_model_idp, df_trainval, df_test, "IDP")

print("IG analysis completed. Results saved in './results_data/'.")

IG analysis completed. Results saved in './results_data/'.
